## Observations

- I have spelled March wrong in data, spelled it Mars...
- Must be correct format to work. Correct format: "Month Year"


In [1]:
!pip install -r requirements.txt
!pip install jupyter-lsp
!pip install 'python-lsp-server[all]'
!pip install "pymongo[srv]"
!%config Completer.use_jedi = False
!ln -s / .lsp_symlink

/bin/bash: line 0: fg: no job control
ln: failed to create symbolic link '.lsp_symlink/': File exists


In [2]:
import numpy as np
import nltk
import pandas as pd
from pandas import json_normalize #add this to req file if needed
import json #add this to req file

from persistence import Connection as Database

In [3]:
from typing import List, Tuple, Dict

In [4]:
# load some data 
db: Database = Database()

Connected to DB


In [5]:
resumes = db.fetch_all_unlabeled_resumes()

In [6]:
import importlib
from util import json_to_df
importlib.reload(json_to_df)

<module 'util.json_to_df' from '/home/jupyter/OpenCV/root/backend/util/json_to_df.py'>

In [7]:
resume_df = json_to_df.create_df(resumes[0])

731 days, 0:00:00
1827 days, 0:00:00


In [8]:
resume_df[0]

,value,label,type,address,website,linkedin,name,phoneNumber,email,github,...,family,company,title,location,fromDate,toDate,description,duration,institution,proficiency
0,Experienced data scientist with an obsession f...,0.0,summary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,contactInfo,"Abraham Lincoln Street 1, Chicago, IL",my-website.com,linkedin.com/in/me123,Monica Music,01012345,monica@music.co.kr,monicamusic123,...,"5 kids, divorced",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.0,experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,SAP,Data Scientist,"Berlin, Berlin, Germany",October 2017,Present,Deep Learning Center of Exellence,62.0,NaN,NaN
3,NaN,0.0,experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Amazon,Software Engineer,"Berlin, Berlin, Germany",March 2017,August 2017,Machine Translation Engineering team,5.0,NaN,NaN
4,NaN,0.0,experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Samsung Electronics,Software Engineer,"Suwon, South Korea",January 2013,anuary 2015,"Computer Vision Lab, DMC Research Center - Com...",12.0,NaN,NaN
5,NaN,0.0,experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"Penta Security Systems, Inc.",SDE Intern,"Seoul, South Korea",February 2012,July 2012,"Database Security Team, Developed Java module ...",5.0,NaN,NaN
6,NaN,0.0,education,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"Master of Engineering (M.Eng.), Computer scien...",Korea,2015,2017,,24.0,Korea Advanced Institute of Science and Techno...,NaN
7,NaN,0.0,education,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"Bachelor's degree, Physics & Computer Science",Korea,2008,2013,,60.0,POSTECH,NaN
8,Data Analysis,0.0,interests,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,0.0,languages,NaN,NaN,NaN,English,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [16]:
from datetime import datetime

def time_of_experience(experience: pd.DataFrame) -> int:
    try:
        fromDate = experience.iloc[0]['fromDate']
        toDate = experience.iloc[0]['toDate']
        present_list = ['today', 'present']
        if toDate.lower() in present_list:
            datetime_object_to = datetime.today()
        else:
            datetime_object_to = datetime.strptime(toDate, '%B %Y')

        datetime_object_from = datetime.strptime(fromDate, '%B %Y')    

        diff = datetime_object_to - datetime_object_from
        seconds_per_month = 86400*30
        months = int(np.floor(diff.total_seconds()/seconds_per_month))
        if months == 0: #one month
            months = 1
        return int(months)
    except ExplicitException:
        months = 12
        return int(months)

In [43]:
experience_df = pd.DataFrame(resume_df[0].iloc[2])

In [44]:
experience_df

,2
value,NaN
label,0.0
type,experience
address,NaN
website,NaN
linkedin,NaN
name,NaN
phoneNumber,NaN
email,NaN
github,NaN


In [45]:
experience_df_transpose = np.transpose(experience_df)

In [46]:
experience_df_transpose

,value,label,type,address,website,linkedin,name,phoneNumber,email,github,...,family,company,title,location,fromDate,toDate,description,duration,institution,proficiency
2,NaN,0.0,experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,SAP,Data Scientist,"Berlin, Berlin, Germany",October 2017,Present,Deep Learning Center of Exellence,62.0,NaN,NaN


In [47]:
experience_df_transpose.iloc[0]['fromDate']

'October 2017'

In [48]:
time_of_experience(experience_df_transpose)

62

In [49]:
def time_since_experience(experience: pd.DataFrame) -> int:
    try:
        toDate = experience.iloc[0]['toDate']
        present_list = ['today', 'present']
        if toDate.lower() in present_list:
            datetime_object_to = datetime.today()
        else:
            datetime_object_to = datetime.strptime(toDate, '%B %Y')
        diff = datetime.today() - datetime_object_to
        seconds_per_month = 86400*30
        months = int(np.floor(diff.total_seconds()/seconds_per_month))
        return int(months)
    except ExplicitException:
        months = 12
        return int(months)

def time_since_education(experience: pd.DataFrame) -> int:
    try:
        toDate = experience.iloc[0]['toDate']
        present_list = ['today', 'present']
        if toDate.lower() in present_list:
            datetime_object_to = datetime.today()
        else:
            datetime_object_to = datetime.strptime(toDate, '%Y')
        diff = datetime.today() - datetime_object_to
        seconds_per_month = 86400*30
        months = int(np.floor(diff.total_seconds()/seconds_per_month))
        return int(months)
    except ExplicitException:
        months = 12
        return int(months)

In [50]:
time_since_experience(experience_df_transpose)

0

0